In [4]:
from transitions import Machine
from transitions.extensions import GraphMachine
from baseball_db_generator import *

In [ ]:
# Get urls and prep info for the url processor
url = "https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2024%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=name-date&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc"
game_urls = GameUrlRetriever(url)
# populate_pitchers_table(url)
# Process urls to get pitches
gup = GameUrlProcessor(game_urls.urls)
gup.process_urls()

In [ ]:
url = "https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2024%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=NYM%7C&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=name-date&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc"#results"
lst = [requests.get(url)]

# Url with results for a certain pitcher from one game
url2= "https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2024%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=NYM%7C&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=name-date&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&type=details&player_id=622663&ep_game_date=2024-08-17&ep_game_pk=745785"

res = requests.get(url=url2)

parser_input = res.text

html_page = BeautifulSoup(parser_input, "html.parser")
rows = html_page.find_all(attrs={"style": "text-align: center;"})
pitch_data = [str(x) for x in rows if "search-pitch-label" in str(x)]
pitch_sequence = [x.split("\">")[2].split("<")[0] for x in pitch_data]
pitch_sequence.reverse()

In [ ]:
# Get urls and prep info for the url processor
url = "https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2024%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=name-date&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc"
game_urls = GameUrlRetriever(url)



In [ ]:
class MarkovChain:
    # to do, convert sequence input to list of ints
    def __init__(self, sequence: list[str]):
        self.sequence = sequence
        self.states = [state for state in set(sequence)]
        self.sequence_length = len(sequence)
        self.transition_counts = self.generate_counts()
        self.transition_probabilities = self.generate_probabilities()
        
    def generate_counts(self):
        states = set(self.sequence)
        state_dict = defaultdict(lambda: defaultdict(int))
        for state1 in states:
            for state2 in states:
                state_dict[state1][state2] = 0
        old_pitch = self.sequence[0]
        
        for i in range(1,len(self.sequence)):
            new_pitch = self.sequence[i]
            state_dict[old_pitch][new_pitch] += 1
            old_pitch = new_pitch
        return state_dict
        
    def generate_probabilities(self):
        transition_probabilities = copy.deepcopy(self.transition_counts)
        
        for state1 in transition_probabilities:
            total_count = sum([x for x in transition_probabilities[state1].values()])
            for state2 in transition_probabilities:
                transition_probabilities[state1][state2] /= total_count
        return transition_probabilities

    def generate_state_machine(self):

        transitions = []
        for state1 in self.states:
            for state2 in self.states:
                if self.transition_probabilities[state1][state2] != 0:
                    transitions.append({"trigger": str(round(self.transition_probabilities[state1][state2],3)), "source": state1, "dest": state2})
        
        pitch_machine = GraphMachine(states=self.states, transitions=transitions, initial=self.sequence[0])
        return pitch_machine

In [ ]:
temp = MarkovChain(pitch_sequence)

In [ ]:
test = temp.generate_state_machine()

In [ ]:
test.get_graph().draw('my_state_diagram.jpg', prog="dot")

In [ ]:
temp.transition_probabilities

In [ ]:
url = "https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2024%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=name-date&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc"
game_urls = GameUrlRetriever(url)

In [ ]:
game_urls.urls[2]

In [ ]:
test = requests.get(game_urls.urls[2]["game_url"])


In [ ]:
page = SoupFactory([test]).convert_to_soup()

In [ ]:
rows = page[0].find_all(attrs={"style": "text-align: center;"})
trs = page[0].find_all("tr")
pitch_data = [str(x) for x in rows if "search-pitch-label" in str(x)]
pitch_sequence = [x.split("\">")[2].split("<")[0] for x in pitch_data]
pitch_sequence.reverse()

In [ ]:

for idx, pitch in enumerate(pitch_sequence):
    pitch_instance = Pitch(pitch_type=pitch, pitcher_id

In [ ]:
page

In [ ]:
trs = page[0].find_all("tr")[1:-1]
pitcher_id_int = int(game_urls.urls[2]["player_id"])
game_id_int = int(game_urls.urls[2]["game_id"])
for idx, tr in enumerate(trs):
    pitch_data = []
    for elem in tr:
        inner_text = elem.text.strip()
        if inner_text != "":
            pitch_data.append(inner_text)
    try:
        pitch_date_str = pitch_data[0]
        pitch_type_str = pitch_data[1]
        pitch_mph_flo = float(pitch_data[2])
        spin_rate_int = int(pitch_data[3])
        pitch_number_int = idx
        pitch_result_str = pitch_data[-2]
        pa_result_str = pitch_data[-1]
        new_pitch = Pitch(
            pitcher_id=pitcher_id_int, 
            game_id=game_id_int,
            pitch_number=pitch_number_int,
            pitch_type=pitch_type_str,
            pitch_result=pitch_result_str,
            pa_result=pa_result_str,
            spin_rate=spin_rate_int,
            pitch_mph=pitch_mph_flo,
            pitch_date=pitch_date_str
        )
        db.add(new_pitch)
        db.commit()
    except ValueError:
        print("Error parsing pitch data. Skipping row")

In [ ]:
len(game_urls.urls)

In [ ]:
pitch_date_str